## Stock market news sentiment analysis

#### Get news urls and full news content from Yahoo finance

In [1]:
import nltk

In [1]:
%pip install newspaper3k
%pip install lxml[html_clean]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
import pandas
from newspaper import Article
import datetime

import numpy as np

import time

In [3]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\vijay\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
api_key = "API KEY"

In [5]:
newsdf = pandas.DataFrame(columns=["category", "stock", "date" ,"title", "summary", "url", "fulltext", "positive", "negative", "total_score", "weight", "sentiment"])

In [ ]:
stock_symbols = {
"Energy" : ["XOM", "CVX", "COP", "SLB", "EOG", "OXY", "PXD", "VLO", "MPC", "PSX"],
# "Materials" : ["DOW", "DD", "LIN", "APD", "SHW", "NEM", "FCX", "ALB", "IP", "ECL"],
# "Industrials" : ["GE", "HON", "UNP", "BA", "MMM", "CAT", "LMT", "RTX", "NOC", "DE"],
# "Consumer Discretionary" : ["AMZN", "TSLA", "HD", "MCD", "NKE", "SBUX", "BKNG", "LOW", "TGT", "GM"],
# "Consumer Staples" : ["PG", "KO", "PEP", "WMT", "COST", "PM", "MO", "MDLZ", "CL", "KMB"],
# "Healthcare" : ["JNJ", "PFE", "MRK", "ABBV", "UNH", "AMGN", "BMY", "GILD", "LLY", "ANTM"],
# "Financials" : ["JPM", "BAC", "WFC", "C", "GS", "MS", "AXP", "USB", "PNC", "SCHW"],
# "Information Technology" : ["AAPL", "MSFT", "GOOGL" , "NVDA", "META", "INTC", "CSCO", "ORCL", "ADBE", "CRM"],
# "Communication Services" : ["GOOGL", "META", "VZ", "T", "CMCSA", "DIS", "NFLX", "CHTR", "TMUS", "ATVI"],
# "Utilities" : ["NEE", "DUK", "SO", "D", "AEP", "EXC", "SRE", "PEG", "ED", "XEL"],
#  "Real Estate" : ["AMT", "PLD", "CCI", "EQIX", "PSA", "SPG", "WELL", "DLR", "O", "AVB"],
}

In [ ]:
for category in list(stock_symbols.keys()):
    print(category)
    for stock_symbol in list(stock_symbols[category]):
        print(stock_symbol)
        # Get news data
        url = f"https://yahoo-finance15.p.rapidapi.com/api/yahoo/ne/news/"+stock_symbol

        headers = {
            "X-RapidAPI-Key": api_key,
            "X-RapidAPI-Host": "yahoo-finance15.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers)
        data = response.json()

        # Extract and print news URLs and descriptions, limit to 5 articles
        for news in data.get("body", [])[:5]:
            print(f"Title: {news['title']}")
            
            url = news['link'].strip()

            try:
                article = Article(url)
                article.download()
                article.parse()
            except:
                continue
            
            title = article.title
            full_text = article.text
            summary = news['description']
            url = article.url
            date = news['pubDate']
            
            new_row = pandas.DataFrame({"category": [category], "stock": [stock_symbol], "date": [date], "title": [title], "summary": [summary], "url": [url], "fulltext": [full_text]})
            newsdf = pandas.concat([newsdf, new_row], ignore_index=True)
            
        time.sleep(0.025)

In [16]:
newsdf.iloc[50:70]

,category,stock,date,title,summary,url,fulltext,positive,negative,total_score,weight,sentiment
50,Materials,DOW,"Thu, 13 Feb 2025 21:15:00 +0000",Dow declares quarterly dividend of 70 cents pe...,Dow (NYSE: DOW) has declared a dividend of 70 ...,https://finance.yahoo.com/news/dow-declares-qu...,"MIDLAND, Mich., Feb. 13, 2025 /PRNewswire/ -- ...",NaN,NaN,NaN,NaN,NaN
51,Materials,DOW,"Sat, 08 Feb 2025 12:06:17 +0000",Earnings Miss: Dow Inc. Missed EPS By 15% And ...,"As you might know, Dow Inc. ( NYSE:DOW ) recen...",https://finance.yahoo.com/news/earnings-miss-d...,"As you might know, Dow Inc. (NYSE:DOW) recentl...",NaN,NaN,NaN,NaN,NaN
52,Materials,DOW,"Fri, 07 Feb 2025 15:11:31 +0000",Is Dow Inc. (DOW) the Best Dividend Stock Yiel...,We recently published a list of 10 Best Divide...,https://finance.yahoo.com/news/dow-inc-dow-bes...,We recently published a list of 10 Best Divide...,NaN,NaN,NaN,NaN,NaN
53,Materials,DOW,"Fri, 07 Feb 2025 11:00:00 +0000",Bear of the Day: Dow Inc. (DOW),The company faces challenges from soft demand ...,https://finance.yahoo.com/news/bear-day-dow-in...,Dow engages in the provision of various materi...,NaN,NaN,NaN,NaN,NaN
54,Materials,DOW,"Tue, 04 Feb 2025 19:54:48 +0000",Dow Inc. (DOW): Among the Large-Cap Stocks Ins...,We recently compiled a list of the 10 Large-Ca...,https://finance.yahoo.com/news/dow-inc-dow-amo...,We recently compiled a list of the 10 Large-Ca...,NaN,NaN,NaN,NaN,NaN
55,Materials,DD,"Thu, 13 Feb 2025 21:38:00 +0000","Top Research Reports for JPMorgan, Costco & SAP",Today's Research Daily features new research r...,https://finance.yahoo.com/news/top-research-re...,"Thursday, February 13, 2025\n\n\n\nThe Zacks R...",NaN,NaN,NaN,NaN,NaN
56,Materials,DD,"Wed, 12 Feb 2025 14:27:00 +0000",Why DuPont de Nemours Inc. (DD) Went Up On Tue...,We recently published a list of These 10 Compa...,https://finance.yahoo.com/news/why-dupont-nemo...,We recently published a list of These 10 Compa...,NaN,NaN,NaN,NaN,NaN
57,Materials,DD,"Wed, 12 Feb 2025 07:06:09 +0000",DuPont de Nemours Inc (DD) Q4 2024 Earnings Ca...,DuPont de Nemours Inc (DD) reports a 7% sales ...,https://finance.yahoo.com/news/dupont-nemours-...,"Revenue: Net sales of $3.1 billion, a 7% incre...",NaN,NaN,NaN,NaN,NaN
58,Materials,DD,"Wed, 12 Feb 2025 05:07:01 +0000",Q4 2024 Dupont De Nemours Inc Earnings Call,Q4 2024 Dupont De Nemours Inc Earnings Call,https://finance.yahoo.com/news/q4-2024-dupont-...,Christopher Mecray; Investor Relations; Dupont...,NaN,NaN,NaN,NaN,NaN
59,Materials,LIN,"Sat, 08 Feb 2025 12:00:34 +0000","Linde Insiders Sell US$7.6m Of Stock, Possibly...","In the last year, many Linde plc ( NASDAQ:LIN ...",https://finance.yahoo.com/news/linde-insiders-...,"In the last year, many Linde plc (NASDAQ:LIN) ...",NaN,NaN,NaN,NaN,NaN


In [11]:
newsdf.to_csv("raw_news.csv", index=False)

In [8]:
newsdf = pandas.read_csv("raw_news.csv")

#### Get news sentiment score from news content using VADER sentiment analysis

In [9]:
def classify_sentiment(score):
    if score > 0.6:
        return "Highly Positive"
    elif score > 0.3:
        return "Moderately Positive"
    elif score > -0.3:
        return "Neutral"
    elif score > -0.6:
        return "Moderately Negative"
    else:
        return "Highly Negative"

In [10]:
sia = SentimentIntensityAnalyzer()

# Initialize weights
weights = [0.5, 0.3, 0.1, 0.05, 0.05]*int(len(newsdf)/5)

# Add columns for sentiment scores
newsdf["positive"] = 0.0
newsdf["negative"] = 0.0
newsdf["total_score"] = 0.0
newsdf["weight"] = weights
newsdf["sentiment"] = ""

# Calculate sentiment scores
for i, row in newsdf.iterrows():
    text = row["fulltext"]
    sentiment = sia.polarity_scores(text)
    
    newsdf.loc[i, "positive"] = float(sentiment["pos"])
    newsdf.loc[i, "negative"] = float(sentiment["neg"])
    newsdf.loc[i, "total_score"] = float(sentiment["compound"])
    
    if sentiment["compound"] > 0.05:
        newsdf.at[i, "sentiment"] = "positive"
    elif sentiment["compound"] < -0.05:
        newsdf.at[i, "sentiment"] = "negative"
    else:
        newsdf.at[i, "sentiment"] = "neutral"

newsdf["weighted_score"] = newsdf["total_score"] * newsdf["weight"]
newsdf.head(10)

ValueError: Length of values (240) does not match length of index (243)

In [ ]:
for category in stock_symbols.keys():
    i = 0
    for stock_symbol in stock_symbols[category]:
        print(f"Stock Symbol: {stock_symbol}")
        
        newsdf_stock = newsdf[newsdf["stock"] == stock_symbol]    
        
        # Calculate final sentiment score
        final_sentiment_score = newsdf_stock["weighted_score"].sum() / newsdf_stock["weight"].sum()
        print(f"Final Weighted Sentiment Score: {final_sentiment_score}")

        # Get sentiment classification
        sentiment_category = classify_sentiment(final_sentiment_score)
        print(f"Stock Sentiment Classification: {sentiment_category}")
        
        newsdf.loc[(stock_symbols[category].index(stock_symbol) + i)*5, "total_weighted_score"] = final_sentiment_score
        newsdf.loc[(stock_symbols[category].index(stock_symbol) + i)*5, "sentiment_category"] = sentiment_category
    i += 1

In [ ]:
newsdf

In [ ]:
newsdf.to_csv("vader_analysis.csv", index=False)